In [1]:
using TuringModels
using Turing

Turing.setadbackend(:reverse_diff);
Turing.turnprogress(false)

d = CSV.read(rel_path("..", "data", "UCBadmit.csv"), delim=';');
size(d) # Should be 12x5

@model m_pois(admit, reject) = begin
   α₁ ~ Normal(0,100)
   α₂ ~ Normal(0,100)

   for i ∈ 1:length(admit)
       λₐ = exp(α₁)
       λᵣ = exp(α₂)
       admit[i] ~ Poisson(λₐ)
       reject[i] ~ Poisson(λᵣ)
   end
end;

posterior = sample(m_pois(d[:admit], d[:reject]), Turing.NUTS(2000, 1000, 0.95));

┌ Info: [Turing]: global PROGRESS is set as false
└ @ Turing /Users/rob/.julia/packages/Turing/UXxKz/src/Turing.jl:24
┌ Info: [Turing] looking for good initial eps...
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/UXxKz/src/inference/support/hmc_core.jl:240
┌ Warning: Numerical error has been found in gradients.
└ @ Turing.Core /Users/rob/.julia/packages/Turing/UXxKz/src/core/ad.jl:168
┌ Warning: grad = [-1755.06, NaN]
└ @ Turing.Core /Users/rob/.julia/packages/Turing/UXxKz/src/core/ad.jl:169
┌ Warning: Numerical error has been found in gradients.
└ @ Turing.Core /Users/rob/.julia/packages/Turing/UXxKz/src/core/ad.jl:168
┌ Warning: grad = [-1755.01, NaN]
└ @ Turing.Core /Users/rob/.julia/packages/Turing/UXxKz/src/core/ad.jl:169
┌ Warning: Numerical error has been found in gradients.
└ @ Turing.Core /Users/rob/.julia/packages/Turing/UXxKz/src/core/ad.jl:168
┌ Warning: grad = [-1755.0, NaN]
└ @ Turing.Core /Users/rob/.julia/packages/Turing/UXxKz/src/core/ad.jl:169
┌ Warning: Nume

Fix the inclusion of adaptation samples

In [2]:
posterior2 = MCMCChain.Chains(posterior.value[1001:2000,:,:], names=posterior.names);

Rethinking/CmdStan result

In [3]:
m_10_yyt_result = "
    mean   sd 5.5% 94.5% n_eff Rhat
 a1 4.99 0.02 4.95  5.02  2201    1
 a2 5.44 0.02 5.41  5.47  2468    1
";

Describe the draws

In [4]:
describe(posterior2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1
Samples per chain = 1000

Empirical Posterior Estimates:
               Mean                  SD                       Naive SE               MCSE          ESS   
  lf_num     0.000000000 0.000000000000000000000000 0.0000000000000000000000000 0.000000000000        NaN
      α₂     5.442012567 0.019197071477433106517774 0.0006070647027374229814933 0.000616455951  969.76357
 elapsed     0.002685179 0.002689088245628723592567 0.0000850364368537309141651 0.000051617626 1000.00000
 epsilon     0.013843264 0.000000000000000013884732 0.0000000000000000004390738 0.000000000000 1000.00000
      α₁     4.984720749 0.024495915615291307537715 0.0007746288671560546150105 0.000690993576 1000.00000
      lp -1301.650191456 1.024868775237942841727090 0.0324091963253907433539780 0.046509437742  485.57279
eval_num    16.216000000 6.332254056924867313682626 0.2002434554272409517050590 0.232929173785  739.04161
  lf_eps     0.013843264 0.000000000000000

End of 10/m_10_yyt.jl#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*